In [ ]:
import pandas as pd
import time
from urllib.error import HTTPError

# Initialize a list to store all data
all_data = []

# Initial delay in seconds
delay = 1

for year in range(1970, 2024):
    # URL of the page with the data
    url = f'https://www.pro-football-reference.com/teams/was/{year}.htm#team_stats'

    while True:
        try:
            # Use pandas to read the tables from the webpage
            tables = pd.read_html(url, header=[0,1])

            # Assuming the table you want is the first one on the page
            df = tables[0]

            # Flatten the multi-index
            df.columns = [' '.join(col).strip() for col in df.columns.values]

            # Select the specified columns
            columns = ["Unnamed: 1_level_0 PF","Unnamed: 2_level_0 Yds","Tot Yds & TO TO","Unnamed: 6_level_0 FL","Unnamed: 7_level_0 1stD","Passing Att","Passing Yds","Passing TD","Passing Int","Passing NY/A","Rushing Att","Rushing Yds","Rushing TD","Rushing Y/A"]
            df = df[columns]

            # Select the last two rows
            df = df[-2:]

            # Append the data to the all_data list
            all_data.append(df)

            # If the request is successful, reset the delay
            delay = 1

            # Exit the loop
            break

        except HTTPError as e:
            if e.code == 429:
                print(f"Too many requests for year {year}. Waiting {delay} seconds.")
                time.sleep(delay)

                # Increase the delay for the next potential 429 error
                delay *= 2
            else:
                print(f"An HTTP error {e.code} occurred for the year {year}.")
                break
        except Exception as e:
            print(f"An error occurred for the year {year}: {e}")
            break

# Concatenate all the dataframes in the list into a single dataframe
all_data_df = pd.concat(all_data)

# Save the DataFrame to a CSV file
all_data_df.to_csv('nfl_team_rankings.csv', index=False)


In [18]:
print(pd.__version__)

2.0.3
